In [4]:
from sparql_dataframe import get
from pandas import *

In [5]:
endpoint = "https://query.wikidata.org/sparql"

## Starting query (the one used for the paper)

In [6]:
s_query = """
SELECT DISTINCT ?writer ?writerLabel ?imdb
WHERE {
  VALUES ?citizen {wd:Q38 wd:Q172579}
  VALUES ?role {wd:Q6625963 wd:Q49757}
  ?writer wdt:P345 ?imdb ;
          rdfs:label ?writerLabel ;
          (wdt:P106 | p:P106 / ps:P106) ?role ;
          (wdt:P27 | p:P27 / ps:P27) ?citizen.
  FILTER (lang(?writerLabel) = 'en')
}
GROUP BY ?writer ?writerLabel ?imdb 
"""

In [7]:
paper_results = get(endpoint, s_query, True).fillna("")
paper_results

,writer,writerLabel,imdb
0,http://www.wikidata.org/entity/Q193018,Gianni Rodari,nm0734427
1,http://www.wikidata.org/entity/Q3751286,Franco Enna,nm0257850
2,http://www.wikidata.org/entity/Q3767161,Giovanni Drovetti,nm0238144
3,http://www.wikidata.org/entity/Q527124,Vincenzo De Crescenzo,nm1876866
4,http://www.wikidata.org/entity/Q3791865,Idolina Landolfi,nm2285763
...,...,...,...
227,http://www.wikidata.org/entity/Q1132008,Goffredo Parise,nm0661650
228,http://www.wikidata.org/entity/Q451312,Umberto Saba,nm0754460
229,http://www.wikidata.org/entity/Q2022,Cesare Pavese,nm0667610
230,http://www.wikidata.org/entity/Q2895923,Beniamino Joppolo,nm0429778


____________________


## `writers_query`
It retrieves every writer, prosaist, novelist or poet that has the Italian citizenship or lived in a country or historical country whose capital city is now an Italian city (to include also writers living, for example, in the Republic of Florence).
Writers are novelists, poets, prosaists or common "writers". However, when an entity is also a screenwriter, it has to be at least also a novelist or a poet.

Problems:
1. Is it better to restrict the threshold of being at least a poet or a novelist as well as a screenwriter excluding the poet possibility (an author to be included should be a novelist if he/she is a screewriter)?
2. When do we set the threshold of being an "Italian writer"? Is it 1200 good?

In [8]:
writers_query = """
SELECT DISTINCT ?writer ?writerLabel ?imdb
WHERE {
  #get writers having an IMDb ID
  ?writer wdt:P345 ?imdb .
  {
    {
      ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P17 ?mcItaly .
  }

  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?writer ?writerLabel ?imdb
    """

wq_post_1200 = """
SELECT DISTINCT ?writer ?writerLabel ?imdb
WHERE {
  #get writers having an IMDb ID and a Date of Born
  ?writer wdt:P345 ?imdb ;
          wdt:P569 ?dob .
  {
    {
      ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P17 ?mcItaly .
  }

  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  FILTER ("1200-01-01"^^xsd:dateTime <= ?dob) 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?writer ?writerLabel ?imdb
"""

In [9]:
writers_df = get(endpoint, writers_query, True).fillna("")
writers_post_1200_df = get(endpoint, wq_post_1200, True).fillna("")

In [10]:
writers_df

,writer,writerLabel,imdb
0,http://www.wikidata.org/entity/Q3645969,Bruno Zanin,nm0953025
1,http://www.wikidata.org/entity/Q558097,Mario Rigoni Stern,nm1167259
2,http://www.wikidata.org/entity/Q1067,Dante Alighieri,nm0019604
3,http://www.wikidata.org/entity/Q3608591,Alberto Spadolini,nm0816553
4,http://www.wikidata.org/entity/Q3615513,Andrea Beltramo,nm1244099
...,...,...,...
732,http://www.wikidata.org/entity/Q95982105,Tinin Mantegazza,nm3060967
733,http://www.wikidata.org/entity/Q1328900,Elia Marcelli,nm0545204
734,http://www.wikidata.org/entity/Q2338386,Stefano D'Arrigo,nm0195499
735,http://www.wikidata.org/entity/Q1889868,Geppi Cucciari,nm2575030


In [11]:
writers_post_1200_df

,writer,writerLabel,imdb
0,http://www.wikidata.org/entity/Q3645969,Bruno Zanin,nm0953025
1,http://www.wikidata.org/entity/Q558097,Mario Rigoni Stern,nm1167259
2,http://www.wikidata.org/entity/Q1067,Dante Alighieri,nm0019604
3,http://www.wikidata.org/entity/Q3608591,Alberto Spadolini,nm0816553
4,http://www.wikidata.org/entity/Q3615513,Andrea Beltramo,nm1244099
...,...,...,...
717,http://www.wikidata.org/entity/Q1042535,Carlo Terron,nm1923000
718,http://www.wikidata.org/entity/Q378674,Arnoldo Foà,nm0289450
719,http://www.wikidata.org/entity/Q993477,Bruno Lauzi,nm0487336
720,http://www.wikidata.org/entity/Q1366593,Paolo Giacometti,nm0316014


## Query for work connections in wikidata

The first query in the first block uses only the IDs of the audiovisual works to retrieve data.

The second uses also the IDs of the writers

In [17]:
work_connections_query = """ 
SELECT DISTINCT ?audioVisualWork ?audioVisualWorkLabel ?imdbAudioVisual
WHERE {
  {
    {
      ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P17 ?mcItaly .
  }

  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  
  
  VALUES ?workType {wd:Q7725634 wd:Q47461344 wd:Q571 wd:Q3331189}   #define what are the possible types of writtenWrok
  ?writtenWork wdt:P50 ?writer ;                #get written wroks written by our writers having the types defined above
               wdt:P31 ?workType .
  
  ?audioVisualWork wdt:P345 ?imdbAudioVisual .  #define audioVisual work as everyting having an imdb ID y now (so to grasp as many products as possile as first)
  {                                             #get aV works derived from the works written by our writers, or inspired by them
    VALUES ?derivedBy {wdt:P737 wdt:P144 wdt:P8371}
    ?audioVisualWork ?derivedBy ?writtenWork .    
  }UNION{
    ?writtenWork wdt:P4969 ?audioVisualWork .    
  }
  
  
  
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?audioVisualWork ?audioVisualWorkLabel ?imdbAudioVisual
"""

In [18]:
work_connections_query_writers_id = """
SELECT DISTINCT ?audioVisualWork ?audioVisualWorkLabel  ?imdbAudioVisual
WHERE {
  #get writers having an IMDb ID
  ?writer wdt:P345 ?imdb .
  {
    {
      ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P17 ?mcItaly .
  }

  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  
  
  VALUES ?workType {wd:Q7725634 wd:Q47461344}   #define what are the possible types of writtenWrok
  ?writtenWork wdt:P50 ?writer ;                #get written wroks written by our writers having the types defined above
               wdt:P31 ?workType .
  
  ?audioVisualWork wdt:P345 ?imdbAudioVisual .  #define audioVisual work as everyting having an imdb ID y now (so to grasp as many products as possile as first)
  {                                             #get aV works derived from the works written by our writers, or inspired by them
    VALUES ?derivedBy {wdt:P737 wdt:P144 wdt:P8371}
    ?audioVisualWork ?derivedBy ?writtenWork .    
  }UNION{
    ?writtenWork wdt:P4969 ?audioVisualWork .    
  }
  
  
  
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?audioVisualWork ?audioVisualWorkLabel ?imdbAudioVisual
"""

In [19]:
work_connections_df = get(endpoint, work_connections_query, True).fillna("")
work_connections_df

,audioVisualWork,audioVisualWorkLabel,imdbAudioVisual
0,http://www.wikidata.org/entity/Q967661,Pinocchio 3000,tt0332318
1,http://www.wikidata.org/entity/Q1425815,Captain Blood,tt0026174
2,http://www.wikidata.org/entity/Q18429,Fellini's Casanova,tt0074291
3,http://www.wikidata.org/entity/Q834460,The End Is My Beginning,tt1529292
4,http://www.wikidata.org/entity/Q683014,The Garden of the Finzi-Continis,tt0065777
...,...,...,...
153,http://www.wikidata.org/entity/Q5638667,Pinocchio,tt0818692
154,http://www.wikidata.org/entity/Q31185402,Filomena Marturano,tt0471700
155,http://www.wikidata.org/entity/Q1130910,Dante's Inferno,tt0026262
156,http://www.wikidata.org/entity/Q3791184,I promessi sposi,tt0172043


In [20]:
work_connections_query_wid_df = get(endpoint, work_connections_query_writers_id, True).fillna("")
work_connections_query_wid_df

,audioVisualWork,audioVisualWorkLabel,imdbAudioVisual
0,http://www.wikidata.org/entity/Q1077585,The Truce,tt0117959
1,http://www.wikidata.org/entity/Q3760834,Geppetto,tt0200656
2,http://www.wikidata.org/entity/Q28214950,La fameuse invasion des ours en Sicile,tt6284064
3,http://www.wikidata.org/entity/Q48967415,The Name of the Rose,tt7572868
4,http://www.wikidata.org/entity/Q27996741,Six Characters in Search of an Author,tt1789909
...,...,...,...
131,http://www.wikidata.org/entity/Q3792653,Il Decamerone,tt0167877
132,http://www.wikidata.org/entity/Q18206120,Corruption in the Palace of Justice,tt4103296
133,http://www.wikidata.org/entity/Q3665312,Caesar the Conqueror,tt0057105
134,http://www.wikidata.org/entity/Q1024861,Cabiria,tt0003740


In [24]:
concat_dfs = concat([work_connections_df, work_connections_query_wid_df])
concat_dfs

,audioVisualWork,audioVisualWorkLabel,imdbAudioVisual
0,http://www.wikidata.org/entity/Q967661,Pinocchio 3000,tt0332318
1,http://www.wikidata.org/entity/Q1425815,Captain Blood,tt0026174
2,http://www.wikidata.org/entity/Q18429,Fellini's Casanova,tt0074291
3,http://www.wikidata.org/entity/Q834460,The End Is My Beginning,tt1529292
4,http://www.wikidata.org/entity/Q683014,The Garden of the Finzi-Continis,tt0065777
...,...,...,...
131,http://www.wikidata.org/entity/Q3792653,Il Decamerone,tt0167877
132,http://www.wikidata.org/entity/Q18206120,Corruption in the Palace of Justice,tt4103296
133,http://www.wikidata.org/entity/Q3665312,Caesar the Conqueror,tt0057105
134,http://www.wikidata.org/entity/Q1024861,Cabiria,tt0003740


#### They do not include different works!